In [14]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from datetime import datetime

In [2]:
tweets = pd.read_json("/Users/sam/Documents/Masters/Data_Systems_Project/Locations/user_locations.json")

In [8]:
def split(x):
    x = str(x).split()[0]
    return x
tweets['created_at'] = tweets['created_at'].map(split)

In [17]:
tweets['created_at'] = pd.to_datetime(tweets['created_at'], format="%Y-%m-%d")

### Import protest data

In [109]:
protest = pd.read_csv('ACLEDNeatAdress.csv')
protest['protest'] = 1
protest = protest.rename(columns={"EVENT_DATE": "created_at", "state2": "state", 'place': 'city'})
protest['created_at'] = pd.to_datetime(protest['created_at'], format="%Y-%m-%d")

In [110]:
splitted=tweets["user_location"].str.split(", ", n = 1, expand = True)
tweets["state"] = splitted[1]
tweets["city"] = splitted[0]

In [111]:
protest = protest.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'LATITUDE', 'LONGITUDE', 'road'])

In [124]:
tweets['created_at'] = tweets['created_at'].astype(str)
protest['created_at'] = protest['created_at'].astype(str)

merged = pd.merge(tweets, protest,  how='left', on=['created_at','state','city'])
merged = merged.fillna(0)

In [125]:
merged.head()

,created_at,id,text,coordinates,lang,place,user_location,state,city,protest
0,2020-05-07,1258365124517203969,This is a sad reality. AmeriKKKa must realize ...,0,en,0,"Detroit, MI",MI,Detroit,0.0
1,2020-05-07,1258365136999518208,RT @JhadiyahG: Please Share Ahmaud Arbery was ...,0,en,0,"Philadelphia, PA",PA,Philadelphia,0.0
2,2020-05-07,1258365210827644930,RT @SharLovegrove: DEVASTATED by the @guardian...,0,en,0,"Crazy Horse, ND",ND,Crazy Horse,0.0
3,2020-05-07,1258365231497203714,RT @ZayyWhosane: This shit so frustrating #Bla...,0,en,0,"New Orleans, LA",LA,New Orleans,0.0
4,2020-05-07,1258365308751970304,We can't break down on a public road with car ...,0,en,"{'id': '7f88808f3d4e1c41', 'url': 'https://api...","Mequon, WI",WI,Mequon,0.0


In [126]:
merged = merged.groupby(['created_at','city']).size().reset_index(name='counts')

In [142]:
temp = merged[['city', 'counts']].groupby('city').mean()
merged['mean'] = temp['counts']

In [148]:
len(temp['counts'])


30588

In [107]:
merged = merged[merged.counts>5]
merged = pd.merge(merged, protest,  how='left', on=['created_at','city'])
merged = merged.fillna(0)

In [108]:
merged.head()

,created_at,city,counts,state,protest
0,2020-05-07,Akron,11,0,0.0
1,2020-05-07,Albany,6,0,0.0
2,2020-05-07,Albuquerque,16,0,0.0
3,2020-05-07,Anaheim,12,0,0.0
4,2020-05-07,Anchorage,6,0,0.0


In [88]:
import statsmodels.api as sm 
    
# defining the dependent and independent variables 
Xtrain = merged[['counts']] 
ytrain = merged[['protest']]
   
# building the model and fitting the data 
log_reg = sm.Logit(ytrain, Xtrain).fit()

Optimization terminated successfully.
         Current function value: 0.693127
         Iterations 3


In [89]:
print(log_reg.summary()) 

                           Logit Regression Results                           
Dep. Variable:                protest   No. Observations:                31756
Model:                          Logit   Df Residuals:                    31755
Method:                           MLE   Df Model:                            0
Date:                Fri, 15 Jan 2021   Pseudo R-squ.:                  -1.692
Time:                        14:38:02   Log-Likelihood:                -22011.
converged:                       True   LL-Null:                       -8175.8
Covariance Type:            nonrobust   LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
counts     -1.873e-05   1.67e-05     -1.123      0.261   -5.14e-05     1.4e-05
